In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import datetime

In [2]:
device = (torch.device('cuda') if torch.cuda.is_available() 
 else torch.device('cpu')) 
print(f"Training on device {device}.")

Training on device cuda.


In [5]:
datapath='F:\\deeplearning\\datasets'

In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(3,16,kernel_size=3,padding=1)
        self.active1=nn.Tanh()
        self.pool1=nn.MaxPool2d(2)
        self.conv2=nn.Conv2d(16,8,kernel_size=3,padding=1)
        self.active2=nn.Tanh()
        self.pool2=nn.MaxPool2d(2)
        self.fullyconnected1=nn.Linear(8*8*8,32)
        self.active3=nn.Tanh()
        self.fullyconnected2=nn.Linear(32,2)
    def forward(self,x):
        input=self.pool1(self.active1(self.conv1(x)))
        input=self.pool2(self.active2(self.conv2(input)))
        input=input.view(-1,8*8*8)
        input=self.active3(self.fullyconnected1(input))
        input=self.fullyconnected2(input)
        return input
        

In [7]:
loaded_model = Net() 
loaded_model.load_state_dict(torch.load(datapath + 'birds_vs_airplanes.pt'))

<All keys matched successfully>

In [8]:
tensor_cifar10=datasets.CIFAR10(datapath,train=False,download=False,transform=transforms.ToTensor())

In [9]:
imgs=torch.stack([img for img ,_ in tensor_cifar10],dim=3)
a = imgs.view(3, -1).mean(dim=1)
b = imgs.view(3, -1).std(dim=1)

In [11]:
transforms_cifar10= datasets.CIFAR10(datapath,train=True,download=False,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(a, b)]))    

In [10]:
transforms_cifar10_val= datasets.CIFAR10(datapath,train=False,download=False,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(a, b)])) 

In [12]:
label_map={0:0,2:1}
cifar2=[ (img,label_map[label]) for img,label in transforms_cifar10 if label in [0,2] ]

In [13]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader): 
    for epoch in range(1, n_epochs + 1): 
        loss_train = 0.0 
        for imgs, labels in train_loader: 
            imgs = imgs.to(device=device) 
            labels = labels.to(device=device) 
            outputs = model(imgs) 
            loss = loss_fn(outputs, labels) 
            optimizer.zero_grad() 
            loss.backward() 
            optimizer.step() 
            loss_train += loss.item() 
    if epoch == 1 or epoch % 10 == 0: 
        print('{} Epoch {}, Training loss {}'.format( datetime.datetime.now(), epoch, loss_train / len(train_loader)))

In [14]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)
model=Net().to(device)
loss_fn = nn.CrossEntropyLoss()
learning_rate=1e-2
optim=optim.SGD(model.parameters(), lr=learning_rate)

In [15]:
training_loop(n_epochs=100,optimizer=optim, model=model, loss_fn=loss_fn, train_loader=train_loader)

2024-08-30 11:00:51.353017 Epoch 100, Training loss 0.15302608076744018
